In [1]:
# !pip install yfinance

In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine,inspect
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
sql_creds = json.load(open("creds.json"))
user = sql_creds["sql_creds"]["user"]
passw = sql_creds["sql_creds"]["passw"]
host = sql_creds["sql_creds"]["host"]
port = 3306
database = 'stock_fundamentals'
# Create an engine instance
engine = create_engine(f'mysql+pymysql://{user}:{passw}@{host}:{port}/{database}')

In [3]:
# stock_df = pd.read_csv("LiveMint.csv")
# piotroski_score_df = pd.read_csv("piotroski_score.csv")
# disclosure_dates_df = pd.read_csv("diclosure_dates_2.csv")

stock_df = pd.read_csv("LiveMint.csv")
piotroski_score_df = pd.read_sql('SELECT * FROM {}'.format("piotroskiscore"), con=engine)
disclosure_dates_df = pd.read_sql('SELECT * FROM {}'.format("boardmeetdate"), con=engine)

In [4]:
stock_df.head()

,Unnamed: 0,stockLink,stockName,stockSymbol
0,0,https://www.livemint.com/market/market-stats/s...,State Bank Of India,SBIN
1,1,https://www.livemint.com/market/market-stats/s...,ICICI Prudential Life Insurance Company,ICICIPRULI
2,2,https://www.livemint.com/market/market-stats/s...,Reliance Industries,RELIANCE
3,3,https://www.livemint.com/market/market-stats/s...,ICICI Bank,ICICIBANK
4,4,https://www.livemint.com/market/market-stats/s...,HDFC Bank,HDFCBANK


In [5]:
piotroski_score_df.head()

,delta_turnover_score,delta_margin_score,delta_eq_score,delta_liquid_score,delta_lev_score,accrual_score,delta_roa_score,cfo_score,roa_score,final_score,stock,date/year
0,1,0,1,1,0,0,1,1,1,6,State Bank Of India,2024
1,1,1,0,1,0,0,1,0,1,5,State Bank Of India,2023
2,0,1,1,1,1,1,1,1,1,8,State Bank Of India,2022
3,0,1,1,0,0,1,0,1,1,5,State Bank Of India,2021
4,1,1,1,0,1,1,1,1,1,8,State Bank Of India,2020


In [6]:
disclosure_dates_df.head()

,tickerId,companyName,remarks,boardMeetDate,purpose
0,S0003016,State Bank Of India,"Inter alia, to seek approval for raising of Lo...",2024-06-19,Others
1,S0003016,State Bank Of India,,2024-05-09,Audited Results & Final Dividend
2,S0003016,State Bank Of India,,2024-02-03,Quarterly Results
3,S0003016,State Bank Of India,,2023-11-04,Quarterly Results
4,S0003016,State Bank Of India,,2023-08-04,Quarterly Results


In [7]:
companyName = disclosure_dates_df["companyName"]
boardMeetDate = disclosure_dates_df["boardMeetDate"]
purpose = disclosure_dates_df["purpose"]

pur = []
date = []
name = []
for p,d,n in zip(purpose,boardMeetDate,companyName):
    if ("Audited" in p): 
        pur.append("Annually")
        date.append(d)
        name.append(n)
    elif ("Quarterly" in p):
        pur.append("Quarterly")
        date.append(d)
        name.append(n)
    else:
        pass

In [8]:
disclosure_dates_df = pd.DataFrame()
disclosure_dates_df["companyName"] =  name
disclosure_dates_df["boardMeetDate"] = date 
disclosure_dates_df["purpose"] = pur

In [9]:
from datetime import datetime
new_dict = {"companyName":[],"disclosure_date":[],"piotroski_score":[],"purpose":[]}
unique_stocks = disclosure_dates_df["companyName"].unique()

for counter,stock in enumerate(unique_stocks):
    print(f'\rProgress: {counter}/{len(unique_stocks)}', end='', flush=True)
    dates = disclosure_dates_df[disclosure_dates_df["companyName"] == stock]
    score = piotroski_score_df[piotroski_score_df["stock"] == stock]
    annual_dates = dates[dates["purpose"] == "Annually"]
    quarterly_dates = dates[dates["purpose"] == "Quarterly"]
    for idx,s in enumerate(score["date/year"]):
        if len(s) == 4 or int(s.split("-")[1]) == 3:
            q = list(annual_dates["boardMeetDate"])
            c = list(annual_dates["companyName"])
            p = list(annual_dates["purpose"])
            a = [i.split("-")[0] for i in list(annual_dates["boardMeetDate"])]
            for i in range(0,len(a)):
                if len(s) == 4 and s == a[i]:
                    new_dict["companyName"].append(c[i])
                    new_dict["disclosure_date"].append(q[i])
                    new_dict["piotroski_score"].append(score.iloc[idx]["final_score"])
                    new_dict["purpose"].append(p[i])
                elif int(s.split("-")[0]) == int(a[i]):
                    
                    new_dict["companyName"].append(c[i])
                    new_dict["disclosure_date"].append(q[i])
                    new_dict["piotroski_score"].append(score.iloc[idx]["final_score"])
                    new_dict["purpose"].append("Quarterly"+"-"+s.split("-")[1])
        else:   
            q = list(quarterly_dates["boardMeetDate"])
            c = list(quarterly_dates["companyName"])
            p = list(quarterly_dates["purpose"])
            month = 0
            for i in range(0,len(q)):
                datetime = datetime.strptime(q[i], '%Y-%m-%d')
                m = datetime.month
                y = datetime.year
                if int(s.split("-")[0]) == int(y):
                    if m > 3 and m <=6:
                        month = 3
                    elif m >6 and m <=9:
                        month = 6
                    elif m > 9 and m <=12:
                        month = 9
                    else:
                        month = 12
                    if int(s.split("-")[1]) == month:
                        new_dict["companyName"].append(c[i])
                        new_dict["disclosure_date"].append(q[i])
                        new_dict["piotroski_score"].append(score.iloc[idx]["final_score"])
                        new_dict["purpose"].append(p[i]+"-"+str(month))
                        
                    else:
                        pass        
final_df = pd.DataFrame(new_dict)     

Progress: 1797/1798

In [10]:
final_df

,companyName,disclosure_date,piotroski_score,purpose
0,State Bank Of India,2024-05-09,6,Annually
1,State Bank Of India,2023-05-18,5,Annually
2,State Bank Of India,2022-05-13,8,Annually
3,State Bank Of India,2021-05-21,5,Annually
4,State Bank Of India,2024-05-09,5,Quarterly-03
...,...,...,...,...
9831,Elecon Engineering Company,2022-05-03,8,Annually
9832,Elecon Engineering Company,2021-05-27,9,Annually
9833,Elecon Engineering Company,2024-04-19,8,Quarterly-03
9834,Elecon Engineering Company,2023-10-20,8,Quarterly-9


In [11]:
import math
import datetime
import time
stockNames = stock_df["stockName"].tolist()
stockSymbols = stock_df["stockSymbol"].tolist()
final_dict = {"stock_name":[],"disclosure_date":[],"piotroski_score":[],"purpose":[],"mean_volume":[],"mean_close":[],"lowest_point_1":[],"stoploss":[],
"1st_exit": [],
"2nd_exit": [],
"3rd_exit": [],
"4th_exit": [],
"5th_exit": [],
"6th_exit": [],
"7th_exit": [],
"8th_exit": [],
"9th_exit": [],
"10th_exit": []}

exit_num = 10
for idx1,name in enumerate(stockNames):
    try:
        print(f'\rProgress: {idx1}/{len(stockNames)} {stockSymbols[idx1]} ', end='', flush=True)
        filtered_df = final_df[final_df["companyName"] == name]
        if len(filtered_df) == 0:
            continue
        disclosure_dates = filtered_df["disclosure_date"].tolist()
        purpose = filtered_df["purpose"].tolist()
        score = filtered_df["piotroski_score"].tolist()
        for idx2,date in enumerate(disclosure_dates):
            returnList = []
            day = int(date.split("-")[2]);month = int(date.split("-")[1]);year = int(date.split("-")[0])
            startDate = datetime.datetime(year, month, day)
            endDate = datetime.datetime(year+1, month, day)
            historical_data = yf.download(stockSymbols[idx1]+".NS", start=startDate, end=endDate, interval='1d', progress=False)
            close =historical_data["Adj Close"].tolist()
            open = historical_data["Open"].tolist()
            numLoop = len(close) / 21
            numLoop = math.floor(numLoop)
            for i in range(0,numLoop):
                try:    
                    returns = ((close[21*(i+1)] - close[0]) / close[0]) * 100
                    returnList.append(returns)
                except Exception as e:
                    pass
                    # print("returns Calculation error")
            if "Annually" in purpose[idx2]:
                if numLoop > 2:
                    lowest_point = min(close[0:63])
                else:
                    lowest_point = min(close)
                if numLoop > 9:
                    stoploss = min(close[0:210])
                else:
                    stoploss = min(close)
            else:
                if numLoop > 0:
                    lowest_point = min(close[0:21])
                else:
                    lowest_point = min(close)
                if numLoop > 2:
                    stoploss = min(close[0:63])
                else:
                    stoploss = min(close)
            final_dict["lowest_point_1"].append(((close[0] - lowest_point)/lowest_point)*100)
            final_dict["stoploss"].append(((close[0] - stoploss)/stoploss)*100)
            padd_num = exit_num - len(returnList)
            if padd_num < 0:
                padd_num = 0
            returnList.extend([0]*padd_num)
            final_dict["mean_volume"].append(np.mean(historical_data["Volume"].tolist()))
            final_dict["mean_close"].append(np.mean(historical_data["Close"].tolist()))
            final_dict["stock_name"].append(name)
            final_dict["disclosure_date"].append(date)
            final_dict["piotroski_score"].append(score[idx2])
            final_dict["purpose"].append(purpose[idx2])
            for i, number in enumerate(returnList[:exit_num]):
                try:
                    key = f"{i+1}th_exit" if i+1 > 3 else f"{i+1}st_exit" if i+1 == 1 else f"{i+1}nd_exit" if i+1 == 2 else f"{i+1}rd_exit"
                    final_dict[key].append(number)
                except Exception as e:
                    print(e)
    
        time.sleep(1)
    except Exception as e:
        print(e)

Progress: 11/1862 PVRINOX L 


1 Failed download:
['PVRINOX.NS']: YFChartError("%ticker%: Data doesn't exist for startDate = 1622572200, endDate = 1654108200")


min() arg is an empty sequence
Progress: 55/1862 SPICEJET K 


1 Failed download:
['SPICEJET.NS']: YFChartError("%ticker%: Data doesn't exist for startDate = 1624991400, endDate = 1656527400")


min() arg is an empty sequence
Progress: 159/1862 UNITDSPR R Partly Paid tly Paid 

...

min() arg is an empty sequence
Progress: 1736/1862 GTECJAINX  


1 Failed download:
['GTECJAINX.NS']: YFChartError("%ticker%: Data doesn't exist for startDate = 1652812200, endDate = 1684348200")


min() arg is an empty sequence
Progress: 1807/1862 SOLEX TI   


1 Failed download:
['SOLEX.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


min() arg is an empty sequence
Progress: 1861/1862 SIIL HLOMB 

In [14]:
from datetime import date
 
# Returns the current local date
today = date.today()
pd.DataFrame(final_dict).to_sql("results", con=engine, if_exists='replace', index=False)
pd.DataFrame(final_dict).to_csv("./Piotroski Score Reports/piotroski_score_{}.csv".format(today))

9618

In [ ]:
pd.DataFrame(final_dict)